In [1]:
import pandas as pd
from datetime import datetime, timedelta, date
import io

ST_2015=pd.read_csv('data/states/Virginia/Virginia2015.csv')
ST_2016=pd.read_csv('data/states/Virginia/Virginia2016.csv')
ST_2017=pd.read_csv('data/states/Virginia/Virginia2017.csv')
ST_2018=pd.read_csv('data/states/Virginia/Virginia2018.csv')
ST_2019=pd.read_csv('data/states/Virginia/Virginia2019.csv')
ST_2020=pd.read_csv('data/states/Virginia/Virginia2020.csv')
us_state_df = ST_2015
us_state_df = pd.concat([us_state_df, ST_2016]).reset_index(drop=True) 
us_state_df = pd.concat([us_state_df, ST_2017]).reset_index(drop=True) 
us_state_df = pd.concat([us_state_df, ST_2018]).reset_index(drop=True)  
us_state_df = pd.concat([us_state_df, ST_2019]).reset_index(drop=True) 
us_state_df = pd.concat([us_state_df, ST_2020]).reset_index(drop=True) 

us_state_df = us_state_df.rename(columns={'DEATH_YEAR': 'Year', 'WEEK_NUM': 'Week', 'FIPS' : 'County',
                                          'NUMBER_OF_DEATHS':'Deaths', 'COUNTY': 'Place of Death County'})

county_pop_df = pd.read_csv("data/states/county_pop.csv", encoding = "utf-8",error_bad_lines=True)
county_pop_df = county_pop_df[county_pop_df['STNAME'] == 'Virginia']

county_fips_df = pd.read_csv("data/states/county_fips.csv", error_bad_lines=True)
county_fips_df = county_fips_df[county_fips_df['STNAME'] == 'Virginia']
county_fips_df["CTYNAME"] = county_fips_df["CTYNAME"].str.upper()
county_fips_dict = dict(zip(county_fips_df.CTYNAME,county_pop_df.COUNTY))


In [2]:

us_state_constant_df = us_state_df.copy()
us_state_constant_df['state_FIPS'] = 51
us_state_constant_df['population_adjustment'] = 'Constant'
us_state_constant_df['pop'] = 0
us_state_constant_df["total_deaths_per100k"] = 0
us_state_constant_df["excess_deaths_per100k"] = 0.0
county_2017_pop= dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2017))
pop_100k = 100000.0
for index, row in us_state_constant_df.iterrows():
    county = row['County']
    pop = county_2017_pop[county]
    us_state_constant_df.at[index,'pop'] = pop
    us_state_constant_df.at[index, 'total_deaths_per100k'] = (row["Deaths"] *pop_100k )/(pop* 1.0)



avg_deaths = us_state_constant_df[us_state_constant_df['Year'] < 2020].groupby(['County', 'Week'])['total_deaths_per100k'].mean()
avg_deaths_df = avg_deaths.to_frame(name = 'avg_deaths').reset_index()

for index, row in us_state_constant_df.iterrows():
    year = row['Year']
    county = row['County']
    week = row['Week']
    if year == 2020:
        avgd = avg_deaths_df[(avg_deaths_df['County'] == county) & (avg_deaths_df['Week'] == week)]
        if not avgd.empty:
            us_state_constant_df.at[index, "excess_deaths_per100k"] = round(row['total_deaths_per100k'] - avgd['avg_deaths'], 4)


In [3]:
county_year_dict = dict()
county_year_dict[2014] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2014))
county_year_dict[2015] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2015))
county_year_dict[2016] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2016))
county_year_dict[2017] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2017))
county_year_dict[2018] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2018))
county_year_dict[2019] = dict(zip(county_pop_df.COUNTY,county_pop_df.POPESTIMATE2019))

us_state_lagged_df = us_state_df.copy()
us_state_lagged_df['state_FIPS'] = 51
us_state_lagged_df['population_adjustment'] = 'Lagged'
us_state_lagged_df['pop'] = 0
us_state_lagged_df["total_deaths_per100k"] = 0
us_state_lagged_df["excess_deaths_per100k"] = 0.0
pop_100k = 100000.0
for index, row in us_state_lagged_df.iterrows():
    year = row['Year'] - 1  
    county = row['County']
    county_pop_dict = county_year_dict[year]
    pop = county_pop_dict[county]
    us_state_lagged_df.at[index,'pop'] = pop
    us_state_lagged_df.at[index, 'total_deaths_per100k'] = (row["Deaths"] *pop_100k )/(pop* 1.0)



avg_deaths = us_state_lagged_df[us_state_lagged_df['Year'] < 2020].groupby(['County', 'Week'])['total_deaths_per100k'].mean()
avg_deaths_df = avg_deaths.to_frame(name = 'avg_deaths').reset_index()

for index, row in us_state_lagged_df.iterrows():
    year = row['Year']
    county = row['County']
    week = row['Week']
    if year == 2020:
        avgd = avg_deaths_df[(avg_deaths_df['County'] == county) & (avg_deaths_df['Week'] == week)]
        if not avgd.empty:
            us_state_lagged_df.at[index, "excess_deaths_per100k"] = round(row['total_deaths_per100k'] - avgd['avg_deaths'],4)       

us_state_merged_df = pd.concat([us_state_constant_df, us_state_lagged_df])
us_state_merged_df = us_state_merged_df.sort_values(['County', 'Week', 'population_adjustment','Year'])


In [4]:
us_state_panel_df = pd.DataFrame(columns=['State_FIPS','County_FIPS','Year', 'Week','Population_Adjustment', 
                                        'Deaths_Per100k', 'Excess_Deaths_Per100k'])        
us_state_panel_df['State_FIPS']  = us_state_merged_df['state_FIPS']
us_state_panel_df['County_FIPS'] = us_state_merged_df['County']
us_state_panel_df['Year']  = us_state_merged_df['Year']
us_state_panel_df['Week'] = us_state_merged_df['Week']
us_state_panel_df['Population_Adjustment'] = us_state_merged_df['population_adjustment']
us_state_panel_df['Deaths_Per100k'] = us_state_merged_df['total_deaths_per100k']
us_state_panel_df['Excess_Deaths_Per100k'] = round(us_state_merged_df['excess_deaths_per100k'],4)

In [5]:
us_state_panel_df.to_csv("virginia_panel.csv",index=False)